In [ ]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import csv
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from skimage.transform import rotate
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing
import pandas as pd
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import matplotlib.patches as patches
import random



batch_size = 32
epochs = 25

# input image dimensions
img_rows, img_cols = 50, 50


def rotation(img, n):
    rotate=iaa.Affine(rotate=(-n, n))
    rotated_image=rotate.augment_image(img)
    return rotated_image

def crop(img, n):
    crop = iaa.Crop(percent=(n, n)) # crop image
    crop_image=crop.augment_image(img)
    return crop_image

def sheer(img, n):
    shear = iaa.Affine(shear=(-n,n))
    shear_image=shear.augment_image(img)
    return shear_image

def hor_flip(img):
    flip_hr=iaa.Fliplr(p=1.0)
    flip_hr_image= flip_hr.augment_image(img)
    return flip_hr_image

def dilate(img, n):
    kernel = np.ones((n,n),np.uint8)
    dilated_image = cv2.dilate(trainxs[800],kernel,iterations = 1)
    return dilate_image

def trans(img, n):
    num = random.randrange(int(n*-1),int(n))
    move=iaa.Affine(translate_percent={"x": num/100}, scale=1)
    trans_image = move.augment_image(img)
    return trans_image

trainxs = []
trainximgs = []
trainys = []

DATADIR = "/home/jupyter/MEng-Project/All Experiment Images/Toy 7 Point 1000"

CATEGORIES = ["1_o", "2_o", "3_o", "4_o","5_o","6_o", "7_o"]

for x,category in enumerate(CATEGORIES):  
    path = os.path.join(DATADIR,category)  
    class_num = CATEGORIES.index(category)
    #class_num = i[x]
    print(class_num)
    

    for img in tqdm(os.listdir(path)):  # iterate over each image per point value
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array 
        img_array = cv2.bitwise_not(img_array)
        img_array = cv2.resize(img_array, (50,50))
        trainys.append(class_num)
        trainxs.append(img_array)
    plt.imshow(img_array, cmap='gray')


            

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
np.set_printoptions(threshold=np.inf)


trainxs1 = np.asarray(trainxs, dtype=np.float32)
trainys1 = np.asarray(trainys, dtype=np.int)
x_train1, x_test1, y_train_old1, y_test_old1 = train_test_split(trainxs1, trainys1, test_size=0.2, random_state=40)



In [ ]:
x_train1.shape

In [ ]:
for i,y in enumerate(x_train1):
    
    new_img_array1 = crop(x_train1[i], 0.1)
    new_img_array1 = np.asarray(new_img_array1, dtype=np.float32)
    x_train1 = np.vstack((x_train1,new_img_array1[None]))
    y_train_old1 = np.append(y_train_old1, y_train_old1[i])
    new_img_array2 = sheer(x_train1[i], 10)
    new_img_array2 = np.asarray(new_img_array2, dtype=np.float32)
    x_train1 = np.vstack((x_train1,new_img_array2[None]))
    y_train_old1 = np.append(y_train_old1, y_train_old1[i])
    new_img_array3 = trans(x_train1[i], 10)
    new_img_array3 = np.asarray(new_img_array3, dtype=np.float32)
    x_train1 = np.vstack((x_train1,new_img_array3[None]))
    y_train_old1 = np.append(y_train_old1, y_train_old1[i])
    new_img_array4 = rotation(x_train1[i], 10)
    new_img_array4 = np.asarray(new_img_array4, dtype=np.float32)
    x_train1 = np.vstack((x_train1,new_img_array4[None]))
    y_train_old1 = np.append(y_train_old1, y_train_old1[i])
    


In [ ]:
y_train1 = to_categorical(y_train_old1)
y_test1 = to_categorical(y_test_old1)

val = 0

x_train = x_train1[:len(x_train1) - int(val*len(x_train1))]
x_test = x_test1[:len(x_test1) - int(val*len(x_test1))]
y_train = y_train1[:len(y_train1) - int(val*len(y_train1))]
y_test = y_test1[:len(y_test1) - int(val*len(y_test1))]
y_train_old = y_train_old1[:len(y_train_old1) - int(val*len(y_train_old1))]

import matplotlib.pyplot as plt
%matplotlib inline
#print(y_train_old)
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})

# Plot Histogram on x
plt.hist(y_train_old, bins=20)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');

x_train.shape

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)


print('x_train shape:', x_train.shape)
x_train = np.array(x_train)
x_test = np.array(x_test)

#print(x_train[3])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Conv2D(128, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Conv2D(356, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(84, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from keras import optimizers
dim_img = img_rows * img_cols
x_train = x_train.reshape(x_train.shape[0], dim_img)
x_test = x_test.reshape(x_test.shape[0], dim_img)
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(dim_img,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(7, activation='softmax'))


sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=sgd,
              metrics=['accuracy'])
#model.summary()

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
trainxs = []
trainys = []

DATADIR = r"C:\Users\Tom\Desktop\All Experiment Images\Toy 7 Point"

CATEGORIES = ["1_o", "2_o", "3_o", "4_o","5_o","6_o", "7_o"]


for x,category in enumerate(CATEGORIES):  
    path = os.path.join(DATADIR,category)  
    class_num = CATEGORIES.index(category)
    #class_num = i[x]
    #print(class_num)
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        img_array = cv2.bitwise_not(img_array)
        img_array = cv2.resize(img_array, (100,100)).flatten()
        trainys.append(class_num)
        trainxs.append(img_array)
        
x_train, x_test, y_train_old, y_test_old = train_test_split(trainxs, trainys, test_size=0.2, random_state=42)

x_train = np.array(x_train)
x_test = np.array(x_test)

#print(x_train[3])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

#import necessary modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#print(x_train)
#create object of the lassifier
neigh = KNeighborsClassifier(n_neighbors=8)
#Train the algorithm
neigh.fit(x_train, y_train_old)
# predict the response
pred = neigh.predict(x_test)
# evaluate accuracy
print ("KNeighbors accuracy score : ",accuracy_score(y_test_old, pred))

# import the necessary packages

In [ ]:
from sklearn.svm import SVC


svm_model_linear = SVC(kernel = 'linear', C = 1).fit(x_train, y_train_old) 
svm_predictions = svm_model_linear.predict(x_test) 

  
# creating a confusion matrix 
#cm = confusion_matrix(y_test_old, svm_predictions) 
print ("svm accuracy score : ", svm_model_linear.score(x_test, y_test_old) )

In [ ]:
# training a DescisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier 


dtree_model = DecisionTreeClassifier(max_depth = 100).fit(x_train, y_train_old) 

pred = dtree_model.predict(x_test) 

print ("Decision Tree accuracy score : ",accuracy_score(y_test_old, pred))

In [ ]:
#from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1, max_features=10, max_depth=15, random_state=0)
gb_clf.fit(x_train, y_train_old)
pred = gb_clf.predict(x_test)
# evaluate accuracy
print ("Gradient Boosted Classifier accuracy score : ",accuracy_score(y_test_old, pred))

In [ ]:
print("Hello world")

In [ ]:
pred = model.predict_classes(x_test)

In [ ]:
print(pred[0])

In [ ]:
from sklearn.metrics import confusion_matrix


cm=confusion_matrix(y_test_old,pred)
print(cm)

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

sn.heatmap(cm,annot=True,cmap='Blues', fmt='g')